In [1]:
import sys
sys.path.append("..")

from src.data_processing.loader import load_dataset, summarize_sales_by_region_year, summarize_models_by_region_year
from src.llm.agent import LLMReportAgent
from src.reporting.markdown_builder import build_markdown_report

In [2]:
# Provide the path to the Excel dataset
dataset_path = "../datasets/BMW sales data (2020-2024).xlsx"

# Load data
df = load_dataset(dataset_path)

# Preprocess
sales_summary = summarize_sales_by_region_year(df, "../reports/sales_summary.json")

model_summary = summarize_models_by_region_year(df, "../reports/models_summary.json")

In [3]:
# Assume summary is your JSON/dict summary of sales
llm_agent = LLMReportAgent()

# # Call analyze with summary and figures directory
# report = llm_agent.analyze(summary, "../reports")

# # Print the analysis report text (with possible plot paths)
# print(report)

In [4]:
sales_report_md = llm_agent.analyze_sales_trend(sales_summary, "../reports")
model_report_md = llm_agent.analyze_model_trend(model_summary, "../reports")

combined_report_path = build_markdown_report(
    [sales_report_md, model_report_md],
    out_dir="../reports",
    report_title="BMW Sales & Model Performance Analysis",
    intro_text="This report combines overall sales trends and model performance highlights."
)

print(f"Combined report saved to: {combined_report_path}")


Combined report saved to: ../reports/report.md
